In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from sklearn import preprocessing, ensemble, metrics, linear_model, model_selection, inspection
import datetime as dt
from scipy import interpolate
from tqdm import tqdm
from pprint import pprint

In [2]:
def merge_covar(df, df_covar, covar_name, on_key='profile_id'):
    df_covar = df_covar[list(df_covar.columns)[:2]]
    df_covar.columns = ['profile_id', covar_name]
    df_merged = pd.merge(left=df, right=df_covar, on=on_key)
    df_merged = df_merged.reset_index(drop=True)
    return df_merged

# Load data

In [3]:
df_profile = pd.read_csv('../datasets/wosis_2019/wosis_201909_profiles.tsv', sep='\t')
print(df_profile.shape)
print(list(df_profile.columns))
df_profile.head(2)

(196498, 23)
['profile_id', 'dataset_id', 'country_id', 'country_name', 'geom_accuracy', 'latitude', 'longitude', 'dsds', 'cfao_version', 'cfao_major_group_code', 'cfao_major_group', 'cfao_soil_unit_code', 'cfao_soil_unit', 'cwrb_version', 'cwrb_reference_soil_group_code', 'cwrb_reference_soil_group', 'cwrb_prefix_qualifier', 'cwrb_suffix_qualifier', 'cstx_version', 'cstx_order_name', 'cstx_suborder', 'cstx_great_group', 'cstx_subgroup']


C:\Users\LZ\AppData\Local\Temp\ipykernel_68980\18200254.py:1: DtypeWarning: Columns (9,10,11,12,14,15,16,17,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_profile = pd.read_csv('../datasets/wosis_2019/wosis_201909_profiles.tsv', sep='\t')


,profile_id,dataset_id,country_id,country_name,geom_accuracy,latitude,longitude,dsds,cfao_version,cfao_major_group_code,...,cwrb_version,cwrb_reference_soil_group_code,cwrb_reference_soil_group,cwrb_prefix_qualifier,cwrb_suffix_qualifier,cstx_version,cstx_order_name,cstx_suborder,cstx_great_group,cstx_subgroup
0,36897,{BE-UplandsI},BE,Belgium,0.000001,50.649889,4.666901,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36898,{BE-UplandsI},BE,Belgium,0.000001,50.583962,4.462114,97.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_chemical = pd.read_csv('../datasets/wosis_2019/wosis_201909_layers_chemical.tsv', sep='\t')
print(df_chemical.shape)
print(list(df_chemical.columns))
df_chemical.head(2)

C:\Users\LZ\AppData\Local\Temp\ipykernel_68980\1985226948.py:1: DtypeWarning: Columns (4,6,8,9,10,11,12,13,15,16,17,18,19,20,22,23,24,25,26,27,29,30,31,32,33,34,36,37,38,39,40,41,43,44,45,46,47,48,50,51,52,53,54,55,57,58,59,60,61,62,64,65,66,67,68,69,71,72,73,74,75,76,78,79,80,81,82,83,85,86,87,88,89,90,92,93,94,95,96,97,99,100,101,102,103,104,106,107,108,109,110,111,113,114,115,116,117,118,120,121,122,123,124,125,127,128,129,130,131,132,134,135,136,137,138,139,141,142,143,144,145,146,148,149,150,151,152) have mixed types. Specify dtype option on import or set low_memory=False.
  df_chemical = pd.read_csv('../datasets/wosis_2019/wosis_201909_layers_chemical.tsv', sep='\t')


(788538, 153)
['profile_id', 'profile_layer_id', 'upper_depth', 'lower_depth', 'layer_name', 'litter', 'tceq_value', 'tceq_value_avg', 'tceq_method', 'tceq_date', 'tceq_dataset_id', 'tceq_profile_code', 'tceq_licence', 'cecph7_value', 'cecph7_value_avg', 'cecph7_method', 'cecph7_date', 'cecph7_dataset_id', 'cecph7_profile_code', 'cecph7_licence', 'cecph8_value', 'cecph8_value_avg', 'cecph8_method', 'cecph8_date', 'cecph8_dataset_id', 'cecph8_profile_code', 'cecph8_licence', 'ecec_value', 'ecec_value_avg', 'ecec_method', 'ecec_date', 'ecec_dataset_id', 'ecec_profile_code', 'ecec_licence', 'elco20_value', 'elco20_value_avg', 'elco20_method', 'elco20_date', 'elco20_dataset_id', 'elco20_profile_code', 'elco20_licence', 'elco25_value', 'elco25_value_avg', 'elco25_method', 'elco25_date', 'elco25_dataset_id', 'elco25_profile_code', 'elco25_licence', 'elco50_value', 'elco50_value_avg', 'elco50_method', 'elco50_date', 'elco50_dataset_id', 'elco50_profile_code', 'elco50_licence', 'elcosp_value',

,profile_id,profile_layer_id,upper_depth,lower_depth,layer_name,litter,tceq_value,tceq_value_avg,tceq_method,tceq_date,...,totc_dataset_id,totc_profile_code,totc_licence,nitkjd_value,nitkjd_value_avg,nitkjd_method,nitkjd_date,nitkjd_dataset_id,nitkjd_profile_code,nitkjd_licence
0,47010,1,0.0,21.0,Ap,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47010,2,21.0,35.0,E1,f,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = pd.read_csv('../datasets/df_socs_0to100.csv')
print(df.shape)
df.head()

(52217, 5)


,profile_id,latitude,longitude,SOCS_0to30,SOCS_30to100
0,36897,50.649889,4.666901,0.422947,0.268229
1,36899,50.597876,4.687607,0.418444,0.423228
2,36901,50.623204,4.466035,0.439610,0.605454
3,36902,50.610517,4.619128,0.397631,0.536603
4,36903,50.598505,4.772798,0.415800,0.497220


In [6]:
colnames_profile = ['profile_id', 'country_id', 'country_name', 'geom_accuracy']
df = pd.merge(left=df_profile[colnames_profile], right=df, on='profile_id', how='right')

# remain the sample points fron the NCSCD dataset
for i in range(len(df)):
    if df['profile_id'][i] < 600:
        df.loc[i, 'geom_accuracy'] = 0.0
df = df[df['geom_accuracy'] < 1/3600].reset_index(drop=True)
df = df.drop('geom_accuracy', axis=1).reset_index(drop=True)

print(df.shape)
df.head(2)

(52217, 7)


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228


In [7]:
df_covar = pd.read_csv('../datasets/covariates/samples_biome.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='biome', on_key='profile_id')
df_covar.groupby(['BIOME_NUM', 'BIOME_NAME']).agg('count')

,,profile_id
BIOME_NUM,BIOME_NAME,
0,,601
1,Tropical & Subtropical Moist Broadleaf Forests,16519
2,Tropical & Subtropical Dry Broadleaf Forests,3825
3,Tropical & Subtropical Coniferous Forests,1348
4,Temperate Broadleaf & Mixed Forests,63905
5,Temperate Conifer Forests,12156
6,Boreal Forests/Taiga,6394
7,"Tropical & Subtropical Grasslands, Savannas & Shrublands",34791
8,"Temperate Grasslands, Savannas & Shrublands",25401


# Biome types recategorize
**Biome types (from Luo et al., 2019, NC)** \
Tropical/subtropical forests [1,2,3]\
Tropical/subtropical grasslands/savannas [7]\
Temperate forests [4,5]\
Temperate grasslands [8]\
Mediterranean/montane shrublands [12]\
Boreal forests [6]\
Tundra [11]\
Deserts [13]\
Croplands

subdivide: [9, 10, 14] -> divided into tropical/temperate/boreal by latitude (split at 23 and 50)\

**Biome types (from Carvalhais et al., 2014, Nature)** \
1 Tropical forests [1,2,3 + 14] \
2 Temperate forests [4,5] \
3 Boreal forests [6] \
4 Tropical savannahs and grasslands [7 + 9,10,12 in -23 ~ 23] \
5 Temperate grasslands and shrublands [8 + 9,10,12 in -50 ~ -23 and 23 ~ 50] \
6 Deserts [13] \
7 Tundra [11] \
8 Croplands [determined by landcover map]\
Wetlands (removed)

subdivide: [9, 10, 14] -> divided into tropical/temperate/boreal by latitude (split at 23 and 50)

In [8]:
df_covar = pd.read_csv('../datasets/covariates/samples_landcover.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='landcover', on_key='profile_id')
df_covar = pd.read_csv('../datasets/covariates/samples_landcover_prop.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='landcover_prop', on_key='profile_id')

print(df.shape)
print(list(df.columns))
df.head()

(52217, 10)
['profile_id', 'country_id', 'country_name', 'latitude', 'longitude', 'SOCS_0to30', 'SOCS_30to100', 'biome', 'landcover', 'landcover_prop']


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0


In [9]:
# adapt from Carvalhais et al., 2014, Nature
biome_type_list = []
biome_type_name_list = []
for i in range(len(df['biome'])):
    biome_type = 0
    biome_type_name = 'NA'
    if df['landcover'][i] in [12]: # or [12,14]
        biome_type = 8
        biome_type_name = 'Croplands'
    elif df['biome'][i] in [1,2,3,14]:
        biome_type = 1
        biome_type_name = 'Tropical forests'
    elif df['biome'][i] in [4,5]:
        biome_type = 2
        biome_type_name = 'Temperate forests'
    elif df['biome'][i] in [6]:
        biome_type = 3
        biome_type_name = 'Boreal forests'
    elif df['biome'][i] in [7] or (df['biome'][i] in [9,10,12] and (df['latitude'][i] >= -23 and df['latitude'][i] <= 23)):
        biome_type = 4
        biome_type_name = 'Tropical savannahs and grasslands'
    elif df['biome'][i] in [8] or (df['biome'][i] in [9,10,12] and ((df['latitude'][i] >= -50 and df['latitude'][i] < -23) or (df['latitude'][i] > 23 and df['latitude'][i] <= 50))):
        biome_type = 5
        biome_type_name = 'Temperate grasslands and shrublands'
    elif df['biome'][i] in [13]:
        biome_type = 6
        biome_type_name = 'Deserts'
    elif df['biome'][i] in [11]:
        biome_type = 7
        biome_type_name = 'Tundra'
    else:
        pass
    biome_type_list.append(biome_type)
    biome_type_name_list.append(biome_type_name)
df['biome_type'] = biome_type_list
df['biome_type_name'] = biome_type_name_list

print(df.shape)
print(list(df.columns))
df.head()

(52217, 12)
['profile_id', 'country_id', 'country_name', 'latitude', 'longitude', 'SOCS_0to30', 'SOCS_30to100', 'biome', 'landcover', 'landcover_prop', 'biome_type', 'biome_type_name']


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop,biome_type,biome_type_name
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0,8,Croplands
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0,8,Croplands
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0,8,Croplands
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0,8,Croplands
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0,8,Croplands


In [10]:
df = df[df['biome_type_name'] != 'NA'].reset_index(drop=True)
print(df.shape)
print(list(np.unique(df['biome_type_name'])))
df.groupby(['biome_type_name']).agg('count')['biome_type']

(52001, 12)
['Boreal forests', 'Croplands', 'Deserts', 'Temperate forests', 'Temperate grasslands and shrublands', 'Tropical forests', 'Tropical savannahs and grasslands', 'Tundra']


biome_type_name
Boreal forests                           937
Croplands                              13881
Deserts                                 2676
Temperate forests                      14810
Temperate grasslands and shrublands     9108
Tropical forests                        4879
Tropical savannahs and grasslands       5324
Tundra                                   386
Name: biome_type, dtype: int64

# Merge the NPP, root mass fraction (RMF) and above- and belowground biomass (AGB, BGB) data

In [11]:
df_covar = pd.read_csv('../datasets/covariates/samples_npp_modis.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='npp_modis', on_key='profile_id')

df_covar = pd.read_csv('../datasets/covariates/samples_rmf.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='rmf', on_key='profile_id')

df_covar = pd.read_csv('../datasets/covariates/samples_agb.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='agb', on_key='profile_id')

df_covar = pd.read_csv('../datasets/covariates/samples_bgb.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='bgb', on_key='profile_id')

print(df.shape)
print(list(df.columns))
df.head()

(52001, 16)
['profile_id', 'country_id', 'country_name', 'latitude', 'longitude', 'SOCS_0to30', 'SOCS_30to100', 'biome', 'landcover', 'landcover_prop', 'biome_type', 'biome_type_name', 'npp_modis', 'rmf', 'agb', 'bgb']


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop,biome_type,biome_type_name,npp_modis,rmf,agb,bgb
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0,8,Croplands,0.629411,62.833527,4.360,1.1100
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0,8,Croplands,0.632558,NaN,4.220,1.0400
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0,8,Croplands,0.595658,NaN,4.390,1.1200
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0,8,Croplands,0.629142,NaN,4.290,1.0700
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0,8,Croplands,0.609726,NaN,5.044,1.5591


In [12]:
df['fbgb'] = df['bgb'] / (df['agb'] + df['bgb'])
df.head()

,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop,biome_type,biome_type_name,npp_modis,rmf,agb,bgb,fbgb
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0,8,Croplands,0.629411,62.833527,4.360,1.1100,0.202925
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0,8,Croplands,0.632558,NaN,4.220,1.0400,0.197719
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0,8,Croplands,0.595658,NaN,4.390,1.1200,0.203267
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0,8,Croplands,0.629142,NaN,4.290,1.0700,0.199627
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0,8,Croplands,0.609726,NaN,5.044,1.5591,0.236116


In [13]:
df = df[~np.isnan(df['npp_modis'])].reset_index(drop=True)
df = df[~np.isnan(df['fbgb'])].reset_index(drop=True)

print(df.shape)
print(list(df.columns))
df.head()

(48557, 17)
['profile_id', 'country_id', 'country_name', 'latitude', 'longitude', 'SOCS_0to30', 'SOCS_30to100', 'biome', 'landcover', 'landcover_prop', 'biome_type', 'biome_type_name', 'npp_modis', 'rmf', 'agb', 'bgb', 'fbgb']


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop,biome_type,biome_type_name,npp_modis,rmf,agb,bgb,fbgb
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0,8,Croplands,0.629411,62.833527,4.360,1.1100,0.202925
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0,8,Croplands,0.632558,NaN,4.220,1.0400,0.197719
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0,8,Croplands,0.595658,NaN,4.390,1.1200,0.203267
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0,8,Croplands,0.629142,NaN,4.290,1.0700,0.199627
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0,8,Croplands,0.609726,NaN,5.044,1.5591,0.236116


# Estimation of the root mass ditribution

## Biome level estimation

In [14]:
df_root = pd.read_csv('../datasets/root_profile_data/root_profiles_D50D95.csv')
print(df_root.shape)
print(list(df_root.columns))
print()
print(len(np.unique(df_root['Vegetation'])))
pprint(list(np.unique(df_root['Vegetation'])))

(564, 31)
['ID', 'Schenk_Jackson_2002', 'latitude', 'longitude', 'Elevation', 'MaxDiameter', 'Measurement', 'Totmass', 'Sampdepth', 'Sampmax', 'Texture', 'Depth_org_horizon', 'Broadleaved_trees', 'Needleleaved_trees', 'Shrubs', 'Semi_shrubs', 'Grasses', 'Forbs', 'Succulents', 'Tree_phenology', 'Anthropogenic', 'Wetland', 'Vegetation', 'UMD_cover', 'PET', 'MAP', 'Rainfall_season', 'D50', 'D95', 'D50_extrapolated', 'D95_extrapolated']

20
['Closed shrubland',
 'DBL forest / boreal',
 'DBL forest / tempera',
 'DBL forest / tropica',
 'EBL forest / tempera',
 'EBL forest / tropica',
 'EBL plantation / tro',
 'ENL forest / boreal',
 'ENL forest / tempera',
 'ENL plantation / tem',
 'Grassland',
 'Mixed forest / tempe',
 'Pasture',
 'Savanna',
 'Tundra',
 'Wooded grassland',
 'Woodland',
 'Xeric forest',
 'alpine herbaceous',
 'open shrubland']


In [15]:
def calc_fr_bnpp(D50, D95, depth_upper, depth_lower):
    c = -1.27875 / (np.log10(D95) - np.log10(D50))
    fr_bnpp = 1 / (1 + (depth_lower / D50)**c) - 1 / (1 + (depth_upper / D50)**c)
    return fr_bnpp

In [16]:
frbnpp_0to30_list = []
frbnpp_30to100_list = []
for i in range(len(df_root)):
    D50 = max(df_root['D50_extrapolated'][i], 0.01)
    D95 = max(df_root['D95_extrapolated'][i], 0.01)
    frbnpp_0to30 = calc_fr_bnpp(D50=D50, D95=D95, depth_upper=0.0, depth_lower=0.3)
    frbnpp_30to100 = calc_fr_bnpp(D50=D50, D95=D95, depth_upper=0.3, depth_lower=1.0)
    frbnpp_0to30_list.append(frbnpp_0to30)
    frbnpp_30to100_list.append(frbnpp_30to100)
df_root['frbnpp_0to30'] = frbnpp_0to30_list
df_root['frbnpp_30to100'] = frbnpp_30to100_list

C:\Users\LZ\AppData\Local\Temp\ipykernel_68980\2225859282.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  fr_bnpp = 1 / (1 + (depth_lower / D50)**c) - 1 / (1 + (depth_upper / D50)**c)


In [17]:
df_root_biome = pd.read_csv('../datasets/root_profile_data/processed/root_samples_biome.csv')
df_root_biome.columns = ['ID', 'biome']
df_root = pd.merge(left=df_root, right=df_root_biome, on='ID')

df_root_landcover = pd.read_csv('../datasets/root_profile_data/processed/root_samples_landcover.csv')
df_root_landcover.columns = ['ID', 'landcover']
df_root = pd.merge(left=df_root, right=df_root_landcover, on='ID')

# print(df_root.shape)
# df_root.head(2)

In [18]:
# adapt from Luo et al., 2019, NC
biome_type_list = []
biome_type_name_list = []
for i in range(len(df_root)):
    biome_type = 0
    biome_type_name = 'NA'
    if df_root['landcover'][i] in [12]: # or [12,14]
        biome_type = 9
        biome_type_name = 'Croplands'
    elif df_root['Vegetation'][i] in ['DBL forest / tropica', 'EBL forest / tropica', 'EBL plantation / tro']:
        biome_type = 1
        biome_type_name = 'Tropical/subtropical forests'
    elif df_root['Vegetation'][i] in ['Grassland', 'Savanna', 'Wooded grassland'] and (df_root['latitude'][i] >= -23 and df_root['latitude'][i] <= 23):
        biome_type = 2
        biome_type_name = 'Tropical/subtropical grasslands/savannas'
    elif df_root['Vegetation'][i] in ['DBL forest / tempera', 'EBL forest / tempera', 'ENL forest / tempera', 'ENL plantation / tem', 'Mixed forest / tempe']:
        biome_type = 3
        biome_type_name = 'Temperate forests'
    elif df_root['Vegetation'][i] in ['Grassland', 'Savanna', 'Wooded grassland'] and ((df_root['latitude'][i] >= -50 and df_root['latitude'][i] < -23) or (df_root['latitude'][i] > 23 and df_root['latitude'][i] <= 50)):
        biome_type = 4
        biome_type_name = 'Temperate grasslands'
    elif df_root['Vegetation'][i] in ['alpine herbaceous'] or df_root['biome'][i] in [10,12]:
        biome_type = 5
        biome_type_name = 'Mediterranean/montane shrublands'
    elif df_root['Vegetation'][i] in ['DBL forest / boreal', 'ENL forest / boreal']:
        biome_type = 6
        biome_type_name = 'Boreal forests'
    elif df_root['Vegetation'][i] in ['Tundra'] or (df_root['Vegetation'][i] in ['Grassland', 'Savanna'] and (df_root['latitude'][i] < -50 or df_root['latitude'][i] > 50)):
        biome_type = 7
        biome_type_name = 'Tundra'
    elif df_root['Vegetation'][i] in ['Xeric forest']:
        biome_type = 8
        biome_type_name = 'Deserts'
    else:
        pass
    biome_type_list.append(biome_type)
    biome_type_name_list.append(biome_type_name)
df_root['biome_type'] = biome_type_list
df_root['biome_type_name'] = biome_type_name_list

print(df_root.shape)
print(list(df_root.columns))
df_root.head(2)

(588, 37)
['ID', 'Schenk_Jackson_2002', 'latitude', 'longitude', 'Elevation', 'MaxDiameter', 'Measurement', 'Totmass', 'Sampdepth', 'Sampmax', 'Texture', 'Depth_org_horizon', 'Broadleaved_trees', 'Needleleaved_trees', 'Shrubs', 'Semi_shrubs', 'Grasses', 'Forbs', 'Succulents', 'Tree_phenology', 'Anthropogenic', 'Wetland', 'Vegetation', 'UMD_cover', 'PET', 'MAP', 'Rainfall_season', 'D50', 'D95', 'D50_extrapolated', 'D95_extrapolated', 'frbnpp_0to30', 'frbnpp_30to100', 'biome', 'landcover', 'biome_type', 'biome_type_name']


,ID,Schenk_Jackson_2002,latitude,longitude,Elevation,MaxDiameter,Measurement,Totmass,Sampdepth,Sampmax,...,D50,D95,D50_extrapolated,D95_extrapolated,frbnpp_0to30,frbnpp_30to100,biome,landcover,biome_type,biome_type_name
0,AC01a,YES,40.05,-105.6,3650,Total,mass,2.46,0.9,no,...,0.12,0.7,0.15,1.25,0.723638,0.209414,17.0,9.0,5,Mediterranean/montane shrublands
1,AC01b,YES,40.05,-105.6,3650,Total,mass,4.15,0.9,yes,...,0.06,0.5,0.06,0.69,0.874385,0.093062,17.0,9.0,5,Mediterranean/montane shrublands


In [19]:
# adapt from Carvalhais et al., 2014, Nature
# 1 Tropical forests [1,2,3 + 14]
# 2 Temperate forests [4,5]
# 3 Boreal forests [6]
# 4 Tropical savannahs and grasslands [7 + 9,10,12 in -23 ~ 23]
# 5 Temperate grasslands and shrublands [8 + 9,10,12 in -50 ~ -23 and 23 ~ 50]
# 6 Deserts [13]
# 7 Tundra [11]
# 8 Croplands [determined by landcover map]

biome_type_list = []
biome_type_name_list = []
for i in range(len(df_root)):
    biome_type = 0
    biome_type_name = 'NA'
    if df_root['landcover'][i] in [12]: # or [12,14]
        biome_type = 8
        biome_type_name = 'Croplands'
    elif df_root['landcover'][i] in [16]:
        biome_type = 6
        biome_type_name = 'Deserts'
    elif df_root['Vegetation'][i] in ['DBL forest / tropica', 'EBL forest / tropica', 'EBL plantation / tro']:
        biome_type = 1
        biome_type_name = 'Tropical forests'
    elif df_root['Vegetation'][i] in ['DBL forest / tempera', 'EBL forest / tempera', 'ENL forest / tempera', 'ENL plantation / tem', 'Mixed forest / tempe']:
        biome_type = 2
        biome_type_name = 'Temperate forests'
    elif df_root['Vegetation'][i] in ['DBL forest / boreal', 'ENL forest / boreal']:
        biome_type = 3
        biome_type_name = 'Boreal forests'
    elif df_root['Vegetation'][i] in ['Grassland', 'Savanna', 'Wooded grassland'] and (df_root['latitude'][i] >= -23 and df_root['latitude'][i] <= 23):
        biome_type = 4
        biome_type_name = 'Tropical savannahs and grasslands'
    elif df_root['Vegetation'][i] in ['Grassland', 'Savanna', 'Wooded grassland'] and ((df_root['latitude'][i] >= -50 and df_root['latitude'][i] < -23) or (df_root['latitude'][i] > 23 and df_root['latitude'][i] <= 50)):
        biome_type = 5
        biome_type_name = 'Temperate grasslands and shrublands'
    elif df_root['Vegetation'][i] in ['Xeric forest']:
        biome_type = 6
        biome_type_name = 'Deserts'
    elif df_root['Vegetation'][i] in ['Tundra'] or (df_root['Vegetation'][i] in ['Grassland', 'Savanna'] and (df_root['latitude'][i] < -50 or df_root['latitude'][i] > 50)):
        biome_type = 7
        biome_type_name = 'Tundra'
    else:
        pass
    biome_type_list.append(biome_type)
    biome_type_name_list.append(biome_type_name)
df_root['biome_type'] = biome_type_list
df_root['biome_type_name'] = biome_type_name_list

print(df_root.shape)
print(list(df_root.columns))
df_root.head(2)

(588, 37)
['ID', 'Schenk_Jackson_2002', 'latitude', 'longitude', 'Elevation', 'MaxDiameter', 'Measurement', 'Totmass', 'Sampdepth', 'Sampmax', 'Texture', 'Depth_org_horizon', 'Broadleaved_trees', 'Needleleaved_trees', 'Shrubs', 'Semi_shrubs', 'Grasses', 'Forbs', 'Succulents', 'Tree_phenology', 'Anthropogenic', 'Wetland', 'Vegetation', 'UMD_cover', 'PET', 'MAP', 'Rainfall_season', 'D50', 'D95', 'D50_extrapolated', 'D95_extrapolated', 'frbnpp_0to30', 'frbnpp_30to100', 'biome', 'landcover', 'biome_type', 'biome_type_name']


,ID,Schenk_Jackson_2002,latitude,longitude,Elevation,MaxDiameter,Measurement,Totmass,Sampdepth,Sampmax,...,D50,D95,D50_extrapolated,D95_extrapolated,frbnpp_0to30,frbnpp_30to100,biome,landcover,biome_type,biome_type_name
0,AC01a,YES,40.05,-105.6,3650,Total,mass,2.46,0.9,no,...,0.12,0.7,0.15,1.25,0.723638,0.209414,17.0,9.0,0,NA
1,AC01b,YES,40.05,-105.6,3650,Total,mass,4.15,0.9,yes,...,0.06,0.5,0.06,0.69,0.874385,0.093062,17.0,9.0,0,NA


In [20]:
df_root = df_root[df_root['biome_type_name'] != 'NA'].reset_index(drop=True)
df_root.groupby(['biome_type_name']).agg('count')['biome_type']

biome_type_name
Boreal forests                          32
Croplands                               35
Deserts                                 23
Temperate forests                      108
Temperate grasslands and shrublands     72
Tropical forests                       118
Tropical savannahs and grasslands       42
Tundra                                  42
Name: biome_type, dtype: int64

In [21]:
df_root_agg = df_root.groupby('biome_type_name', as_index=False).agg({'frbnpp_0to30': [np.mean, np.std], 'frbnpp_30to100': [np.mean, np.std]})

# replace columns
cols_new = []
cols_ori = df_root_agg.columns
for col_ori in cols_ori[1:]:
    col_new = '{}_{}'.format(col_ori[0], col_ori[1])
    cols_new.append(col_new)
df_root_agg.columns = [cols_ori[0][0]] + cols_new

df_root_agg.iloc[[5, 3, 0, 6, 4, 2, 7, 1]]

,biome_type_name,frbnpp_0to30_mean,frbnpp_0to30_std,frbnpp_30to100_mean,frbnpp_30to100_std
5,Tropical forests,0.788476,0.159602,0.167515,0.114993
3,Temperate forests,0.663798,0.179684,0.277442,0.150242
0,Boreal forests,0.861832,0.114975,0.116198,0.089569
6,Tropical savannahs and grasslands,0.621654,0.247419,0.277062,0.181081
4,Temperate grasslands and shrublands,0.682321,0.218335,0.251218,0.177482
2,Deserts,0.689312,0.210560,0.243236,0.154079
7,Tundra,0.911134,0.091176,0.074597,0.071360
1,Croplands,0.661807,0.220014,0.246373,0.157348


In [22]:
df = pd.merge(left=df, right=df_root_agg, on='biome_type_name').reset_index(drop=True)

In [23]:
print(df.shape)
print(list(df.columns))
df.head()

(48557, 21)
['profile_id', 'country_id', 'country_name', 'latitude', 'longitude', 'SOCS_0to30', 'SOCS_30to100', 'biome', 'landcover', 'landcover_prop', 'biome_type', 'biome_type_name', 'npp_modis', 'rmf', 'agb', 'bgb', 'fbgb', 'frbnpp_0to30_mean', 'frbnpp_0to30_std', 'frbnpp_30to100_mean', 'frbnpp_30to100_std']


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop,...,biome_type_name,npp_modis,rmf,agb,bgb,fbgb,frbnpp_0to30_mean,frbnpp_0to30_std,frbnpp_30to100_mean,frbnpp_30to100_std
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0,...,Croplands,0.629411,62.833527,4.360,1.1100,0.202925,0.661807,0.220014,0.246373,0.157348
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0,...,Croplands,0.632558,NaN,4.220,1.0400,0.197719,0.661807,0.220014,0.246373,0.157348
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0,...,Croplands,0.595658,NaN,4.390,1.1200,0.203267,0.661807,0.220014,0.246373,0.157348
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0,...,Croplands,0.629142,NaN,4.290,1.0700,0.199627,0.661807,0.220014,0.246373,0.157348
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0,...,Croplands,0.609726,NaN,5.044,1.5591,0.236116,0.661807,0.220014,0.246373,0.157348


## Estimation RMF by the global estimated map

In [24]:
df_covar = pd.read_csv('../datasets/root_profile_data/processed/samples_frbnpp_0to30_mean.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='frbnpp_0to30', on_key='profile_id')

df_covar = pd.read_csv('../datasets/root_profile_data/processed/samples_frbnpp_30to100_mean.csv')
df = merge_covar(df=df, df_covar=df_covar, covar_name='frbnpp_30to100', on_key='profile_id')

print(df.shape)
print(list(df.columns))
df.head()

(48557, 23)
['profile_id', 'country_id', 'country_name', 'latitude', 'longitude', 'SOCS_0to30', 'SOCS_30to100', 'biome', 'landcover', 'landcover_prop', 'biome_type', 'biome_type_name', 'npp_modis', 'rmf', 'agb', 'bgb', 'fbgb', 'frbnpp_0to30_mean', 'frbnpp_0to30_std', 'frbnpp_30to100_mean', 'frbnpp_30to100_std', 'frbnpp_0to30', 'frbnpp_30to100']


,profile_id,country_id,country_name,latitude,longitude,SOCS_0to30,SOCS_30to100,biome,landcover,landcover_prop,...,rmf,agb,bgb,fbgb,frbnpp_0to30_mean,frbnpp_0to30_std,frbnpp_30to100_mean,frbnpp_30to100_std,frbnpp_0to30,frbnpp_30to100
0,36897,BE,Belgium,50.649889,4.666901,0.422947,0.268229,4,12.0,31.0,...,62.833527,4.360,1.1100,0.202925,0.661807,0.220014,0.246373,0.157348,0.627012,0.338206
1,36899,BE,Belgium,50.597876,4.687607,0.418444,0.423228,4,12.0,31.0,...,NaN,4.220,1.0400,0.197719,0.661807,0.220014,0.246373,0.157348,0.639976,0.331678
2,36901,BE,Belgium,50.623204,4.466035,0.439610,0.605454,4,12.0,31.0,...,NaN,4.390,1.1200,0.203267,0.661807,0.220014,0.246373,0.157348,0.570951,0.350751
3,36902,BE,Belgium,50.610517,4.619128,0.397631,0.536603,4,12.0,31.0,...,NaN,4.290,1.0700,0.199627,0.661807,0.220014,0.246373,0.157348,0.627012,0.338206
4,36903,BE,Belgium,50.598505,4.772798,0.415800,0.497220,4,12.0,31.0,...,NaN,5.044,1.5591,0.236116,0.661807,0.220014,0.246373,0.157348,0.656594,0.339052


# Calculate soil carbon turnover time

In [25]:
# using biome-level frbnpp
df['bnpp'] = df['npp_modis'] * (df['fbgb'])
df['tovr_0to30'] = df['SOCS_0to30'] / (df['bnpp'] * df['frbnpp_0to30_mean'])
df['tovr_30to100'] = df['SOCS_30to100'] / (df['bnpp'] * df['frbnpp_30to100_mean'])

df = df[df['tovr_0to30'] > 0].reset_index(drop=True)
df = df[df['tovr_30to100'] > 0].reset_index(drop=True)

df['tovr_0to30_log'] = np.log10(df['tovr_0to30'])
df['tovr_30to100_log'] = np.log10(df['tovr_30to100'])
print(df.shape)

(48170, 28)


In [26]:
# using global estimated map of frbnpp
df['bnpp'] = df['npp_modis'] * (df['fbgb'])
df['tovr_0to30'] = df['SOCS_0to30'] / (df['bnpp'] * df['frbnpp_0to30'])
df['tovr_30to100'] = df['SOCS_30to100'] / (df['bnpp'] * df['frbnpp_30to100'])

df = df[df['tovr_0to30'] > 0].reset_index(drop=True)
df = df[df['tovr_30to100'] > 0].reset_index(drop=True)

df['tovr_0to30_log'] = np.log10(df['tovr_0to30'])
df['tovr_30to100_log'] = np.log10(df['tovr_30to100'])
print(df.shape)

(47897, 28)


In [28]:
# remove samples with extremely high SOCS (the values larger than the largest value predicted in SoilGrid2.0)
df = df[df['SOCS_0to30'] <= 36.7].reset_index(drop=True)
df = df[df['SOCS_30to100'] <= 85.7].reset_index(drop=True)
df = df[df['bnpp'] >= 0.01].reset_index(drop=True)
df = df[df['frbnpp_0to30'] >= 0.01].reset_index(drop=True)
df = df[df['frbnpp_30to100'] >= 0.01].reset_index(drop=True)
df['tovr_sub2top'] = df['tovr_30to100'] / df['tovr_0to30']
df = df[df['tovr_0to30'] <= 5000].reset_index(drop=True)
df = df[df['tovr_0to30'] >= 1].reset_index(drop=True)
df = df[df['tovr_30to100'] <= 10000].reset_index(drop=True)
df = df[df['tovr_30to100'] >= 1].reset_index(drop=True)

print(df.shape)

(46237, 29)
